In [1]:
import os
import pandas as pd
from urbansim.utils import misc
import orca

In [2]:
orca.add_injectable("store", pd.HDFStore(os.path.join(misc.data_dir(),
                                                     "semcog_data.h5"), mode="r"))

In [4]:
@orca.table('annual_household_control_totals')
def annual_household_control_totals(store):
    df = store['annual_household_control_totals']
    return df

@orca.table('households')
def households(store):
    df = store['households']
    return df
    
@orca.table('persons')
def persons(store):
    df = store['persons']
    return df

In [10]:
import numpy as np, pandas as pd
from urbansim.models import transition, relocation
@orca.step('households_transition')
def households_transition(households, persons, annual_household_control_totals, iter_var):
    ct = annual_household_control_totals.to_frame()
    for col in ct.columns:
        i = 0
        if col.endswith('_max'):
            if len(ct[col][ct[col]==-1]) > 0:
                ct[col][ct[col]==-1] = np.inf
                i+=1
            if i > 0:
                ct[col] = ct[col] + 1
    tran = transition.TabularTotalsTransition(ct, 'total_number_of_households')
    model = transition.TransitionModel(tran)
    hh = households.to_frame(households.local_columns)
    p = persons.to_frame(persons.local_columns)
    new, added_hh_idx, new_linked = \
        model.transition(hh, iter_var,
                         linked_tables={'linked': (p, 'household_id')})
    new.loc[added_hh_idx, "building_id"] = -1
    orca.add_table("households", new)
    orca.add_table("persons", new_linked['linked'])

In [11]:
orca.run([
    "households_transition",  # households transition
#     "households_relocation",  # households relocation model
], iter_vars=range(2012, 2014))

Running iteration 1 with iteration value 2012
Running step 'households_transition'
Time to execute step 'households_transition': 136.04 s
Total time to execute iteration 1 with iteration value 2012: 136.04 s
Running iteration 2 with iteration value 2013
Running step 'households_transition'
Time to execute step 'households_transition': 117.70 s
Total time to execute iteration 2 with iteration value 2013: 117.70 s


/home/da/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
print orca.get_injectable('iter_var')
hh_sim = orca.get_table('households').to_frame()

store = orca.get_injectable('store')
hh_base = store['households']

2013
